In [1]:
%run "..\Startup_py3.py"
sys.path.append(r"..\..\..\Documents")

import ImageAnalysis3 as ia
%matplotlib notebook

from ImageAnalysis3 import *
print(os.getpid())

# library design specific tools
from ImageAnalysis3.library_tools import LibraryDesigner as ld
from ImageAnalysis3.library_tools import LibraryTools as lt
# biopython imports
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio.Blast import NCBIXML

13064


In [2]:
# load names
readout_folder = r'\\10.245.74.212\Chromatin_NAS_2\Libraries\Readouts'
sel_filename = os.path.join(readout_folder, r'20-mer_barcode_screen.xlsx')
import pandas as pd
sel_df = pd.read_excel(sel_filename, )
print(sel_df)

         name  color  brightness              sequence
0     NDB_784    750    5.975574  ATGCCACTCGGATATTTCTA
1     NDB_826    750    5.914067  GTCTAAGGACATCGACATAT
2     NDB_376    750    5.576824  ATCAACGTACCCACAAAGTC
3     NDB_865    750    5.548981  TCACATATCCGACCGGGATC
4     NDB_817    750    5.469553  AATGCTTAGTCAACGAGTTC
..        ...    ...         ...                   ...
449  NDB_1071    561    3.042401  GGTTCTGCACCCGGTAGGTC
450   NDB_576    561    3.039261  CAGCCGATGACGGGAAACGC
451   NDB_870    561    3.037168  CTTAATATCTCGGCGGTCGT
452   NDB_468    561    3.025657  CGAGCTCATAGCCATACGAA
453   NDB_978    561    3.024198  ATAGTAAAGGCGATGCATTG

[454 rows x 4 columns]


In [3]:
# load readouts
NDBs = []
for _record in SeqIO.parse(os.path.join(readout_folder, 'updated_NDBs.fasta'), "fasta"):
    NDBs.append(_record)


In [4]:
sel_rds = {_ch:[] for _ch in np.unique(sel_df['color'])}
for _rd_name, _ch in zip(sel_df['name'], sel_df['color']):
    _match = [_rd for _rd in NDBs if _rd.id == _rd_name]
    if len(_match) == 1:
        sel_rds[_ch].append(_match[0])

In [6]:
# save to file
for _ch, _rds in sel_rds.items():
    _save_filename = os.path.join(readout_folder, f"highest_snr_{_ch}.fasta")
    with open(_save_filename, 'w') as _output_handle:
        SeqIO.write(_rds, _output_handle, "fasta")

In [8]:
# select top NDBs that higher than NDB292
kept_rds = {_ch:[] for _ch in sel_rds}
for _ch, _rds in sel_rds.items():
    for _rd in _rds:
        _ndb_id = int(_rd.id.split('NDB_')[1])
        if _ndb_id > 630:
            kept_rds[_ch].append(_rd)

In [9]:
# save to file
for _ch, _rds in kept_rds.items():
    _save_filename = os.path.join(readout_folder, f"snr_after_630_{_ch}.fasta")
    with open(_save_filename, 'w') as _output_handle:
        SeqIO.write(_rds, _output_handle, "fasta")